A classifier to classify images of dogs and cats
In the implementation the use of ImageFolder to load data
is demonstrated

In [ ]:
#Import PyTorch Libraries
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms,datasets

#Import data manipulation libraries
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt

#Import PyTorch data processing libraries
from torch.utils.data import DataLoader
from torch.utils.data import Subset


In [ ]:
#Create transforms
#Create a transform with resizing
transform_train = transforms.Compose([
    transforms.Resize([64,64]),
    transforms.ToTensor(),
])


In [ ]:
#create path to dataset
data_dir ='D:/ML4HST_2023/Datasets/DogsCats'
#Create train dataset
train_dataset = datasets.ImageFolder(data_dir+'\Train',transform=transform_train)
val_dataset = datasets.ImageFolder(data_dir+'\Val',transform=transform_train)

print('Train Set: ',len(train_dataset))
print('Valid Set: ',len(val_dataset))
print(train_dataset[0][0].shape)


In [ ]:
#Create datloaders for training and validation data sets
batch_size = 32

torch.manual_seed(1)
train_dl = DataLoader(train_dataset, batch_size, shuffle=True)
valid_dl = DataLoader(val_dataset, batch_size, shuffle=False)
print(len(train_dl))
print(len(valid_dl))


In [ ]:
#Display Sample Images
samples, labels = next(iter(DataLoader(train_dataset, batch_size=4, shuffle=True)))
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:4])
np_grid_imgs = grid_imgs.numpy()
#Display the shape of the grid containing four images each of size 64x64. The black border will increase the size.
print(np_grid_imgs.shape)
#imshow of matplotlib requires the images in the format(WHC)
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))
plt.show()


In [ ]:
#Build the CNN Architecture
model = nn.Sequential()

model.add_module('conv1', nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1))
model.add_module('relu1', nn.ReLU())        
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))  

model.add_module('conv2', nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1))
model.add_module('relu2', nn.ReLU())        
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))   

model.add_module('conv3', nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1))
model.add_module('relu3', nn.ReLU())        
model.add_module('pool3', nn.MaxPool2d(kernel_size=2))   

model.add_module('conv4', nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1))
model.add_module('relu4', nn.ReLU())
#Add a average pooling
model.add_module('pool4', nn.AvgPool2d(kernel_size=8))


In [ ]:
#Flatten the output of the last pooling layer
model.add_module('flatten', nn.Flatten())


In [ ]:
#Add a fully connected output layer with a single neuron
model.add_module('fc', nn.Linear(256, 1)) 
model.add_module('sigmoid', nn.Sigmoid()) 


In [ ]:
#Display the model details
model

In [ ]:
#setup to use GPU
device = torch.device("cuda:0")
model = model.to(device)


In [ ]:
#Use Binary Cross Entropy loss function 
loss_fn = nn.BCELoss()
#Use Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
#Define a training function
def train(model, num_epochs, train_dl, valid_dl):
    loss_hist_train = [0] * num_epochs
    accuracy_hist_train = [0] * num_epochs
    loss_hist_valid = [0] * num_epochs
    accuracy_hist_valid = [0] * num_epochs
    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_dl:
            x_batch = x_batch.to(device) 
            y_batch = y_batch.to(device) 
            pred = model(x_batch)[:, 0]
            loss = loss_fn(pred, y_batch.float())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item()*y_batch.size(0)
            is_correct = ((pred>=0.5).float() == y_batch).float()
            accuracy_hist_train[epoch] += is_correct.sum().cpu()

        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)
        
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                x_batch = x_batch.to(device) 
                y_batch = y_batch.to(device) 
                pred = model(x_batch)[:, 0]
                loss = loss_fn(pred, y_batch.float())
                loss_hist_valid[epoch] += loss.item()*y_batch.size(0) 
                is_correct = ((pred>=0.5).float() == y_batch).float()
                accuracy_hist_valid[epoch] += is_correct.sum().cpu()

        loss_hist_valid[epoch] /= len(valid_dl.dataset)
        accuracy_hist_valid[epoch] /= len(valid_dl.dataset)
        
        print(f'Epoch {epoch+1} accuracy: {accuracy_hist_train[epoch]:.4f} val_accuracy: {accuracy_hist_valid[epoch]:.4f}')
    return loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid


In [ ]:
#Begin Training
torch.manual_seed(1)
num_epochs = 20
hist = train(model, num_epochs, train_dl, valid_dl)


In [ ]:
x_arr = np.arange(len(hist[0])) + 1

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist[0], '-o', label='Train loss')
ax.plot(x_arr, hist[1], '--<', label='Validation loss')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)

ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist[2], '-o', label='Train acc.')
ax.plot(x_arr, hist[3], '--<', label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)

#plt.savefig('figures/14_17.png', dpi=300)
plt.show()


In [ ]:
#Evaluate model performance with test data
test_dataset = datasets.ImageFolder(data_dir+'\Test',transform=transform_train)
print('Test Set: ',len(test_dataset))
test_dl = DataLoader(test_dataset,batch_size, shuffle=False)
accuracy_test = 0.0

#Set the model for evaluation using the model on GPU
model.eval()
with torch.no_grad():
    for x_batch,y_batch in test_dl:
        x_batch = x_batch.to(device) 
        y_batch = y_batch.to(device)
        pred = model(x_batch)[:,0]
        is_correct = ((pred>=0.5).float() == y_batch).float()
        accuracy_test += is_correct.sum()

accuracy_test /= len(test_dataset)
print('Test Accuracy: {0:.4f}'.format(accuracy_test))
        

In [ ]:
#Display the results for a small subset of test data with their probabilities
model = model.to('cpu')
samples, labels = next(iter(DataLoader(test_dataset, batch_size=32, shuffle=True)))
pred = model(samples)[:,0]*100
fig = plt.figure(figsize=(15,7))
for j in range(10,20):
    ax = fig.add_subplot(2,5,j-10+1)
    ax.set_xticks([]);ax.set_yticks([])
    ax.imshow(samples[j].permute(1,2,0))
    if labels[j] == 1:
        label = 'dog'
    else:
        label = 'cat'
    ax.text(
        0.5,-0.15,
        f'GT:{label:s}\nPr({label:s})={pred[j]:.0f}%',
        size = 16,
        horizontalalignment='center',
        verticalalignment='center',
        transform=ax.transAxes
    )
plt.show()